1. Data Preparation¶
To use BigQuery, we need to import Python package below:

In [1]:
from google.cloud import bigquery 
import plotly.graph_objs as go
import plotly.express as px
import json 

# Create a "Client" object
crime = bigquery.Client()

dataset_ref = crime.dataset("london_crime", project = "bigquery-public-data")

# Construct a reference
table_ref = dataset_ref.table("crime_by_lsoa")

# API request, request to fetch the table
table = crime.get_table(table_ref)

Using Kaggle's public dataset BigQuery integration.


In [2]:
# Preview the first 10 rows of the table
crime.list_rows(table, max_results =10).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000478,Brent,Sexual Offences,Rape,0,2008,1
1,E01000619,Brent,Sexual Offences,Rape,1,2008,1
2,E01000619,Brent,Sexual Offences,Rape,0,2008,7
3,E01000156,Barnet,Sexual Offences,Rape,0,2008,11
4,E01000156,Barnet,Sexual Offences,Rape,0,2008,7
5,E01000384,Bexley,Sexual Offences,Rape,1,2008,1
6,E01001301,Ealing,Sexual Offences,Rape,1,2008,1
7,E01001295,Ealing,Sexual Offences,Rape,0,2008,1
8,E01000291,Barnet,Sexual Offences,Rape,0,2008,8
9,E01000630,Brent,Sexual Offences,Rape,1,2008,2


#Data Exploration and Visualisation 
Which borough has the most crime?

In [3]:
#Find out borough with most crime
borough_count = """
                SELECT borough, SUM(value) AS num_crimes
                FROM `bigquery-public-data.london_crime.crime_by_lsoa` 
                GROUP BY borough ORDER BY num_crimes DESC
                """
borough_count = crime.query(borough_count).result().to_dataframe()
borough_count

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough,num_crimes
0,Westminster,455028
1,Lambeth,292178
2,Southwark,278809
3,Camden,275147
4,Newham,262024
5,Croydon,260294
6,Ealing,251562
7,Islington,230286
8,Tower Hamlets,228613
9,Brent,227551


In [4]:
fig = px.bar(borough_count, x= "num_crimes", y="borough", title ="Borough in Greater Area of London with Most Crimes") 
fig.update_layout(yaxis={'categoryorder':'total ascending'})

#Minor Category Analysis

In [5]:
#Find out most minor crime category that happened in Greater London Area
minor_crime_count1 = """
              SELECT DISTINCT minor_category,major_category, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa` 
              GROUP BY minor_category,major_category ORDER BY num_crimes DESC
              """
minor_crime_count1 = crime.query(minor_crime_count1).result().to_dataframe()
minor_crime_count1

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,major_category,num_crimes
0,Other Theft,Theft and Handling,980085
1,Theft From Motor Vehicle,Theft and Handling,569956
2,Burglary in a Dwelling,Burglary,491282
3,Harassment,Violence Against the Person,458124
4,Assault with Injury,Violence Against the Person,451001
5,Possession Of Drugs,Drugs,431948
6,Common Assault,Violence Against the Person,413690
7,Theft From Shops,Theft and Handling,345142
8,Other Theft Person,Theft and Handling,308842
9,Criminal Damage To Motor Vehicle,Criminal Damage,265463


In [6]:
fig = px.bar(minor_crime_count1, x= "num_crimes", y="minor_category", title ="Minor Category Crimes in Greater Area of London") 
fig.update_layout(yaxis={'categoryorder':'total ascending'})

Major Crime Category Analysis

In [7]:
#Find out most major crime category that happened in Greater London Area
major_crime_count1 = """
              SELECT DISTINCT major_category, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa` 
              GROUP BY major_category ORDER BY num_crimes DESC
              """
major_crime_count1 = crime.query(major_crime_count1).result().to_dataframe()
major_crime_count1

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,major_category,num_crimes
0,Theft and Handling,2661861
1,Violence Against the Person,1558081
2,Burglary,754293
3,Criminal Damage,630938
4,Drugs,470765
5,Robbery,258873
6,Other Notifiable Offences,106349
7,Fraud or Forgery,5325
8,Sexual Offences,1273


In [8]:
fig = px.bar(major_crime_count1, x= "num_crimes", y="major_category", title ="Major Category Crimes in Greater Area of London") 
fig.update_layout(yaxis={'categoryorder':'total ascending'})

We Want to Focus on Crimes Figure in The City of London

In [9]:
#Find out most minor crime category that happened in Greater London Area
minor_crime_count_london = """
              SELECT DISTINCT minor_category,major_category, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London'
              GROUP BY minor_category,major_category ORDER BY num_crimes DESC
              """
minor_crime_count_london = crime.query(minor_crime_count_london).result().to_dataframe()
minor_crime_count_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,major_category,num_crimes
0,Other Theft,Theft and Handling,270
1,Other Theft Person,Theft and Handling,194
2,Theft/Taking of Pedal Cycle,Theft and Handling,41
3,Harassment,Violence Against the Person,36
4,Assault with Injury,Violence Against the Person,29
5,Possession Of Drugs,Drugs,28
6,Common Assault,Violence Against the Person,27
7,Personal Property,Robbery,23
8,Theft From Shops,Theft and Handling,22
9,Theft From Motor Vehicle,Theft and Handling,18


In [10]:
fig = px.bar(minor_crime_count_london, x= "num_crimes", y="minor_category", title ="Minor Category Crimes in The City of London") 
fig.update_layout(yaxis={'categoryorder':'total ascending'})

Major Crime in the Ciry of London

In [11]:
#Find out crime that happened per Months in the City of London in year 2008 - 2016
crime_month_london = """
              SELECT month, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London'
              GROUP BY month 
              ORDER BY month
              """
crime_month_london = crime.query(crime_month_london).result().to_dataframe()
crime_month_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,month,num_crimes
0,1,58
1,2,56
2,3,49
3,4,53
4,5,61
5,6,52
6,7,53
7,8,75
8,9,90
9,10,70


In [12]:
#Find out crime that happened per Months in the City of London in year 2016
crime_month_london = """
              SELECT month, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London' AND year = 2016
              GROUP BY month 
              ORDER BY month
              """
crime_month_london = crime.query(crime_month_london).result().to_dataframe()
crime_month_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,month,num_crimes
0,1,10
1,2,10
2,3,12
3,4,15
4,5,9
5,6,8
6,7,13
7,8,19
8,9,19
9,10,17
